In [1]:
%reload_ext autoreload
%autoreload 2

import os
import sys
import pickle

sys.path.append("..")
from dm21cm.dm_params import DMParams

sys.path.append(os.environ['DH_DIR'])
from darkhistory.main import evolve as evolve_DH
from darkhistory.config import load_data

In [2]:
tf_version = 'zf01'

load_data('dep_tf', prefix=f"{os.environ['DH_DATA_DIR']}/../DHdata_v1_1_{tf_version}")
load_data('ics_tf', prefix=f"{os.environ['DH_DATA_DIR']}/../DHdata_v1_1_{tf_version}");

****** Loading transfer functions... ******
Using data at /n/holyscratch01/iaifi_lab/yitians/darkhistory/DHdata_v1_1/../DHdata_v1_1_zf01
    for propagating photons...  

 Done!
    for low-energy photons...  Done!
    for low-energy electrons...  Done!
    for high-energy deposition...  Done!
    for total upscattered CMB energy rate...  Done!
****** Loading complete! ******
****** Loading transfer functions... ******
    for inverse Compton (Thomson)...  Done!
    for inverse Compton (relativistic)...  Done!
    for inverse Compton (energy loss)...  Done!
****** Loading complete! ******


In [3]:
# phph
dm_params = DMParams(
    mode = 'decay',
    primary = 'phot_delta',
    m_DM = 1e8, # [eV]
    lifetime = 1e25, #[s]
)

In [3]:
# ee
dm_params = DMParams(
    mode = 'decay',
    primary = 'elec_delta',
    m_DM = 1e8, # [eV]
    lifetime = 1e25, #[s]
)

In [3]:
# off
dm_params = DMParams(
    mode = 'decay',
    primary = 'phot_delta',
    m_DM = 1e8, # [eV]
    lifetime = 1e50, #[s]
)

In [4]:
run_name = 'xc_xrayST'
tf_version = 'zf01'

soln = evolve_DH(
    DM_process=dm_params.mode, mDM=dm_params.m_DM,
    primary=dm_params.primary,
    sigmav=dm_params.sigmav, lifetime=dm_params.lifetime,
    struct_boost=dm_params.struct_boost,
    start_rs=3000, end_rs=4.+1, coarsen_factor=12, verbose=1,
    # use fake reionization to get case-A recomb. coeff.
    reion_switch=True, reion_rs=47.,
    photoion_rate_func=[lambda x: 0., lambda x: 0., lambda x: 0.],
    photoheat_rate_func=[lambda x: 0., lambda x: 0., lambda x: 0.],
    cross_check_21cmfast=True,
    cross_check_21cmfast_tf_version=tf_version,
    debug_inject_ST_xray=True,
)

pickle.dump(soln, open(f"{os.environ['DM21CM_DIR']}/outputs/dh/{run_name}_soln.p", 'wb'))

Loading time: 0.026 s


  0%|          | 0/534 [00:00<?, ?it/s]WARNING:root:Cross checking 21cmfast!


Initialization time: 0.066 s


 65%|██████▌   | 349/534 [00:52<00:11, 15.52it/s]

xrayST: injecting 3.414e-24 eV/Bavg
xrayST: injecting 7.925e-24 eV/Bavg
xrayST: injecting 1.867e-23 eV/Bavg
xrayST: injecting 4.245e-23 eV/Bavg
xrayST: injecting 9.392e-23 eV/Bavg
xrayST: injecting 2.033e-22 eV/Bavg
xrayST: injecting 4.532e-22 eV/Bavg
xrayST: injecting 9.989e-22 eV/Bavg
xrayST: injecting 2.145e-21 eV/Bavg


...

xrayST: injecting 1.065e-01 eV/Bavg
xrayST: injecting 1.114e-01 eV/Bavg
xrayST: injecting 1.163e-01 eV/Bavg


571it [01:03, 14.56it/s]

xrayST: injecting 1.214e-01 eV/Bavg
xrayST: injecting 1.265e-01 eV/Bavg
xrayST: injecting 1.317e-01 eV/Bavg


573it [01:03,  9.04it/s]

xrayST: injecting 1.370e-01 eV/Bavg
xrayST: injecting 1.424e-01 eV/Bavg
Main loop time: 63.368 s
